# Visualização Cartográfica

_“A criação de mapas é um dos esforços intelectuais mais antigos da humanidade e também um dos mais complexos, com teoria científica, representação gráfica, fatos geográficos e considerações práticas misturados em uma variedade infinita de maneiras.”_ &mdash; [H. J. Steward](https://books.google.com/books?id=cVy1Ms43fFYC)

Cartografia &ndash; o estudo e a prática da criação de mapas &ndash; tem uma rica história que abrange séculos de descobertas e desenvolvimento de design. A visualização cartográfica utiliza técnicas de mapeamento para representar dados contendo informações espaciais, como localizações, rotas ou trajetórias na superfície da Terra.

<div style="float: right; margin-left: 1em; margin-top: 1em;"><img width="300px" src="https://gist.githubusercontent.com/jheer/c90d582ef5322582cf4960ec7689f6f6/raw/8dc92382a837ccc34c076f4ce7dd864e7893324a/latlon.png" /></div>

Aproximando a Terra como uma esfera, podemos denotar posições usando um sistema de coordenadas esféricas de _latitude_ (ângulo em graus ao norte ou sul do _equator_) e _longitude_ (ângulo em graus que especifica a posição leste-oeste). Nesse sistema, um _paralelo_ é um círculo de latitude constante, enquanto um _meridiano_ é um círculo de longitude constante. O [_meridiano principal_](https://en.wikipedia.org/wiki/Prime_meridian) está localizado a 0° de longitude e, por convenção, é definido como aquele que passa pelo Observatório Real de Greenwich, na Inglaterra.

Para "achatarmos" uma esfera tridimensional em um plano bidimensional, é necessário aplicar uma [projeção](https://en.wikipedia.org/wiki/Map_projection) que converta pares de coordenadas (`longitude`, `latitude`) em coordenadas (`x`, `y`). Assim como as [escalas](https://github.com/uwdata/visualization-curriculum/blob/master/altair_scales_axes_legends.ipynb), as projeções transformam um domínio de dados (posição espacial) em um intervalo visual (posição em pixels). No entanto, enquanto as escalas que vimos até agora operam em um domínio unidimensional, as projeções cartográficas são, por natureza, bidimensionais.

Neste notebook, apresentaremos os fundamentos da criação de mapas e da visualização de dados espaciais com Altair, incluindo:

- Formatos de dados para representar elementos geográficos,
- Técnicas de geovisualização, como mapas de pontos, símbolos e coropléticos, e
- Uma revisão das projeções cartográficas mais comuns.

_Este notebook faz parte do [currículo de visualização de dados.](https://github.com/uwdata/visualization-curriculum)._

In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data

## Dados Geográficos: GeoJSON e TopoJSON

Até agora, trabalhamos com *datasets* em JSON e CSV, que correspondem à tabelas de dados compostas de linhas (*records*) e colunas (*fields*). Para que possamos representar regiões geográficas (países, estados, etc.) e trajetórias (rotas de voo, linhas de metrô, _etc_.), precisamos expandir nosso repertório com novas fortamações capazes de trabalhar com formas geométricas diversas.

O [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) consegue modelar propriedades geográficas utilizando um formato JSON especializado. Uma *propriedade* (`feature`) em um GeoJSON pode incluir informações geográficas &ndash; por exemplo, as coordenadas de `longitude` e `latitude` correspondentes à fronteira de um país &ndash; bem como atributos adicionais dos dados.

Aqui está um objeto `feature` do GeoJSON correspondente à divisa do estado americano do Colorado:

~~~ json
{
  "type": "Feature",
  "id": 8,
  "properties": {"name": "Colorado"},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [[-106.32056285448942,40.998675790862656],[-106.19134826714341,40.99813863734313],[-105.27607827344248,40.99813863734313],[-104.9422739227986,40.99813863734313],[-104.05212898774828,41.00136155846029],[-103.57475287338661,41.00189871197981],[-103.38093099236758,41.00189871197981],[-102.65589358559272,41.00189871197981],[-102.62000064466328,41.00189871197981],[-102.052892177978,41.00189871197981],[-102.052892177978,40.74889940428302],[-102.052892177978,40.69733266640851],[-102.052892177978,40.44003613055551],[-102.052892177978,40.3492571857556],[-102.052892177978,40.00333031918079],[-102.04930288388505,39.57414465707943],[-102.04930288388505,39.56823596836465],[-102.0457135897921,39.1331416175485],[-102.0457135897921,39.0466599009048],[-102.0457135897921,38.69751011321283],[-102.0457135897921,38.61478847120581],[-102.0457135897921,38.268861604631],[-102.0457135897921,38.262415762396685],[-102.04212429569915,37.738153927339205],[-102.04212429569915,37.64415206142214],[-102.04212429569915,37.38900413964724],[-102.04212429569915,36.99365914927603],[-103.00046581851544,37.00010499151034],[-103.08660887674611,37.00010499151034],[-104.00905745863294,36.99580776335414],[-105.15404227428235,36.995270609834606],[-105.2222388620483,36.995270609834606],[-105.7175614468747,36.99580776335414],[-106.00829426840322,36.995270609834606],[-106.47490250048605,36.99365914927603],[-107.4224761410235,37.00010499151034],[-107.48349414060355,37.00010499151034],[-108.38081766383978,36.99903068447129],[-109.04483707103458,36.99903068447129],[-109.04483707103458,37.484617466122884],[-109.04124777694163,37.88049961001363],[-109.04124777694163,38.15283644441336],[-109.05919424740635,38.49983761802722],[-109.05201565922046,39.36680339854235],[-109.05201565922046,39.49786885730673],[-109.05201565922046,39.66062637372313],[-109.05201565922046,40.22248895514744],[-109.05201565922046,40.653823231326896],[-109.05201565922046,41.000287251421234],[-107.91779872584989,41.00189871197981],[-107.3183866123281,41.00297301901887],[-106.85895696843116,41.00189871197981],[-106.32056285448942,40.998675790862656]]
    ]
  }
}
~~~

Este `feature` possui um objeto `properties`, que pode incluir uma quantidade arbitrária de _data fields_, e um objeto `geometry`, que, neste caso, contém um único polígono. Este, por sua vez, consiste de coordenadas `[longitude, latitude]` para a fronteira do estado. As coordenadas continuam para o lado direito da tela (arraste para o lado para descobrir...).

Para aprender mais sobre os mínimos detalhes do GeoJSON, veja a [especificação oficial do GeoJSON](http://geojson.org/) ou leia a [cartilha auxiliar de Tom MacWright](https://macwright.org/2015/03/23/geojson-second-bite).

Uma desvantagem do GeoJSON como forma de armazenamento é que ele pode conter redundâncias, resultando em arquivos maiores do que o necessário. Por exemplo: Colorado faz divisa com seis outros estados (ou sete, se você contar o vértice compartilhado com o Arizona). Ao invés de utilizarmos listas independetes de coordenadas para cada um destes estados (o que resultaria em diversas interseções), podemos guardar a informação de forma mais compacta salvando cada divisa compartilhada uma única vez, representando a _topologia_ de regiões geográficas. Felizmente, o formato [TopoJSON](https://github.com/topojson/topojson/blob/master/README.md) faz exatamente isso!

Vamos carregar um arquivo TopoJSON dos países do mundo (em uma resolução de 110 metros):

In [2]:
world = data.world_110m.url
world

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json'

In [3]:
world_topo = data.world_110m()

In [4]:
world_topo.keys()

dict_keys(['type', 'transform', 'objects', 'arcs'])

In [5]:
world_topo['type']

'Topology'

In [6]:
world_topo['objects'].keys()

dict_keys(['land', 'countries'])

_Dê uma olhada no objeto dicionário do TopoJSON (`world_topo`) para ver o seu conteúdo._

Nos dados acima, a propriedade `objects` indica os objetos nomeados que podemos extrair: geometrias de todos os países (`countries`) ou um único polígono representando toda a massa de terra (`land`) do planeta. Ambos podem ser descompactados em arquivos GeoJSON que podemos visualizar.

Como o TopoJSON é um formato especializado, precisamos ensinar o Altair à analisar o formato TopoJSON, indicando qual objeto (elemento nomeado) queremos extrair da topologia. O código a seguir indica que queremos extrair propriedades GeoJSON do _dataset_ `world` para o objeto `countries`:

~~~ js
alt.topo_feature(world, 'countries')
~~~

O método `alt.topo_feature` que chamamos acima expande para o seguinte Vega-Lite JSON:

~~~ json
{
  "values": world,
  "format": {"type": "topojson", "feature": "countries"}
}
~~~

Agora que conseguimos carregar dados geográficos, estamos prontos para começar a criar mapas!

## Marcadores *Geoshape*

Para visualizar dados geográficos, o Altair fornece o tipo de marcador `geoshape`. Para criar um mapa simples, podemos criar um marcador `geoshape` e dar a ele os nossos dados em TopoJSON, que então serão descompactados em propriedades GeoJSON, uma para cada país do mundo:

In [7]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape()

alt.Chart(...)

No exemplo acima, o Altair aplicar as configurações padrão de cor (azul) e de projeção (`mercator`). Podemos customizar as cores das regiões e a espessura das fronteiras utilizando propriedades padrão de marcadores. Utilizando o método `project`, podemos também modificar a projeção cartográfica:

In [8]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator'
)

alt.Chart(...)

Por padrão, o Altair ajusta automaticamente a projeção para que todos os dados caibam dentro do comprimento e altura do gráfico. Podemos também especificar os parâmetros, como `scale` (nível de zoom) e `translate` (translação), para customizar as configurações da projeção. A seguir, ajustamos as parâmetros `scale` e `translate` para focar na Europa:

In [9]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator', scale=400, translate=[100, 550]
)

alt.Chart(...)

_Note que a resolução dos dados de 110 metros se torna notável nesta escala. Para visualizar litorais e fronteiras com mais detalhes, precisamos de um arquivo de entrada com geometrias mais refinadas. Ajuste os parâmetros `scale` e `translate` para focar em outras regiões do mapa!_

Até então, o nosso mapa mostra apenas países. Usando o operador `layer`, podemos combinar múltiplos elementos do mapa. O Altair conta com _geradores de dados_ que podemos usar para criar dados para camadas de mapa adicionais:

- O operador *sphere* (`{'sphere': True}`) fornece uma representação em GeoJSON de todas a esféra do planeta. Podemos criar um marcador `geoshape` adicional para preencher o formato da Terra em uma camada de fundo.
- O gerador de *gráticulas* (`{'graticule': ...}`) cria uma propriedade GeoJSON representando uma gratícula: uma grade formada por linhas de latitude e de longitude. Por padrão, a gratícula possui meridianos e paralelos a cada 10° entre ±80° de latitude. Nos polos, temos meridianos a cada 90°. Essas configurações podem ser customizadas usando as propriedades `stepMinor` e `stepMajor`.

Vamos juntar camadas *sphere*, *graticule* e marcadores de país para criar uma especificação de mapa reutilizável:

In [10]:
map = alt.layer(
    # use the sphere of the Earth as the base layer
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # add a graticule for geographic reference lines
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # and then the countries of the world
    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
        fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
    )
).properties(
    width=600,
    height=400
)

Podemos estender o mapa com uma projeção desejada e visualizar o resultado. A seguir, aplicamos a [Projeção Natural da Terra](https://en.wikipedia.org/wiki/Natural_Earth_projection). A camada _sphere_ fornece o fundo azul-claro e a camada _graticule_ fornece as linhas brancas de referência geográfica.

In [11]:
map.project(
    type='naturalEarth1', scale=110, translate=[300, 200]
).configure_view(stroke=None)

alt.LayerChart(...)

## Mapas de pontos

Além dos dados _geométricos_ dados pelos arquivos GeoJSON ou TopoJSON, muitas bases de dados incluem informações geográficas em campos de cordenadas `longitude` e `latitute`, ou tomando referência em regiões geográficas, como por exemplo o nome de países, estados, cartões postais, _etc_., que podem ser mapeadas em coordenadas usando um [serviço de gecodificação](https://en.wikipedia.org/wiki/Geocoding). Em alguns casos, Em alguns casos, os dados de localização são ricos o suficiente para que possamos ver padrões significativos projetando apenas os pontos de dados —  &mdash; sem precisar de nenhuma projeção de mapa!

Vamos analisar um conjunto de dados de códigos postais de 5 dígitos nos Estados Unidos, incluindo coordenadas de `longitude`, `latitude` para cada agência postal, além de um campo `zip_code`.

In [ ]:
zipcodes = data.zipcodes.url
zipcodes

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/zipcodes.csv'

Podemos ver cada agência dos correios usando uma pequena (1-pixel)  marca `square` (quadrada). No entanto, para indicar as posições, _não_ usamos os canais `x` e `y`. _Por que isso?_

Enquando as projeções cartográficas mapeiam as coordenadas (`longitude`, `latitude`) em coordenadas (`x`, `y`), elas podem fazê-lo de formas arbitrárias. Não há garantia, por exemplo, que `longitude` → `x` e `latitude` → `y`! Em vez disso, o Altair inclui canais especiais de codificação `longitude` e `latitude` para lidar com coordenadas geográficas. Esses canais indicam quais campos de dados devem ser mapeados nas coordenadas `longitude` e `latitude`, e então aplica uma projeção para mapear essas coordenadas em posições (`x`, `y`).

In [31]:
alt.Chart(zipcodes).mark_square(
    size=1, opacity=1
).encode(
    longitude='longitude:Q', # aplique o campo chamado 'longitude no canal longitude'
    latitude='latitude:Q'    # aplique o campo chamado 'latitude no canal latitude'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Traçando apenas códigos postais, podemos ver o contorno dos Estados Unidos e extrair padrões interessantes acerca da densidade de agências de correios, sem um mapa base ou elementos de referência adicionais._

Usamos a projeção `albersUsa`, que toma algumas liberdades com a geometria real da Terra, com versões em escala do Alasca e do Havaí no canto inferior esquerdo. Como não especificamos os parâmetros de projeção `scale` ou `translate`, o Altair automaticamente os configura para serem projetados.

Agora podemos fazer mais perguntas sobre nossa base de dados. Por exemplo, ela tem alguma lógica ou razão para a alocação dos códigos postais? Para responder essa pergunta, podemos adicionar uma codificação de cores com base no primeiro dígito do código postal. Primeiro adicionamos uma transformação `calculate` para extrair o primeiro dígito, e então codificamos o resultado usando o canal de cores:

In [32]:
alt.Chart(zipcodes).transform_calculate(
    digit='datum.zip_code[0]'
).mark_square(
    size=2, opacity=1
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Para dar um zoom em um dado em específico, adicione uma transformação de filtro para limitar os dados mostrados! Tente adicionar uma [seleção interativa](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb) para filtrar em um único dígito e dinamicamente atualizar o mapa. E se atente para usar strings (\`'1'\`) ao invés de números (\`1\`) ao filtrar os valores dos dígitos!_

(Esse exemplo é inspirado na clássica visualização de [zipdecode](https://benfry.com/zipdecode/), por Ben Fry!)

Podemos ainda nos perguntar o que a sequência de códigos postais pode indicar. Uma maneira de explorar essa questão é conectar cada código postal usando uma marca `line`, como feito na visualização [ZipScribble](https://eagereyes.org/zipscribble-maps/united-states), por Robert Kosara.

In [33]:
alt.Chart(zipcodes).transform_filter(
    '-150 < datum.longitude && 22 < datum.latitude && datum.latitude < 55'
).transform_calculate(
    digit='datum.zip_code[0]'
).mark_line(
    strokeWidth=0.5
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N',
    order='zip_code:O'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Agora podemos ver como os códigos postais se aglomeram em áreas menores, indicando uma alocação hierárquica dos códigos por localização, mas com uma notável variação por entre as aglomerações locais._

Se você estava atento para nossos mapas anteriores, deve ter notado que temos códigos postais sendo marcados no canto superior esquerdo! Esses correspondem a territórios como Porto Rico ou Samoa Americana, que contêm códigos postais americanos, mas são mapeadas como coordenadas `null` (`0`, `0`) pela projeção `albersUsa`. Além disso, o Alasca e o Havaí podem complicar nossa visualização das ligações de segmentos. Em resposta, o código acima inclui um filtro adicional que remove pontos fora dos nossos limites escolhidos de `longitude` e `latitude`.

_Remova os filtros acima e veja o que acontece!_

## Mapas de Símbolos

Agora, vamos combinar um mapa base e dados plotados como camadas separadas. Vamos examinar a rede estadunidense de voos comerciais, considerando aeroportos e rotas aéreas. Para fazer isso, vamos precisar de três bases de dados. Para nosso mapa base, vamos usar um arquivo TopoJSON para os Estados Unidos com uma resolução de 10m, contendo as características para estados (`states`)  ou países (`countries`):

In [34]:
usa = data.us_10m.url
usa

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json'

Para os aeroportos, vamos usar uma base de dados com campos para as coordenadas de `longitude` e `latitude` de cada aeroporto, além do código `iata` de cada aeroporto &mdash; por exemplo, `'SEA'` para o  [Aeroporto Internacional de Seattle-Tacoma](https://en.wikipedia.org/wiki/Seattle%E2%80%93Tacoma_International_Airport).

In [35]:
airports = data.airports.url
airports

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/airports.csv'

Por fim, vamos usar uma base de dados que contém os campos origem (`origin`) e (`destination`) com os códigos IATA dos aeroportos correspondentes:

In [36]:
flights = data.flights_airport.url
flights

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/flights-airport.csv'

Vamos começar criando um mapa base usando a projeção `albersUsa`, e adicionar uma camada que plote marcas `circle` para cada aeroporto:

In [37]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(airports).mark_circle(size=9).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip='iata:N'
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_Quantos aeroportos! Obviamente, nem todos eles são grandes hubs._

Assim como a nossa base de dados de códigos postais, nossos dados de aeroportos incluem pontos que ficam fora do território continental dos Estados Unidos. Então, novamente podemos ver pontos no canto superior esquerdo. Talvez seja de nossa vontade filtrar esses pontos, mas antes disso, precisamos saber mais sobre eles.

_Atualize a projeção do mapa para `albers` &ndash; evitando o comportamento idiossincrático de `albersUsa` &ndash; para que as localizações verdadeiras desses pontos adicionais sejam mostradas!_

Agora, ao invés de mostrar todos os aeroportos da mesma maneira, vamos identificar os maiores pelo número de rotas que saem de cada aeroporto. Vamos usar a base de dados rotas (`routes`) como nossa principal fonte de dados: ela contém uma lista de rotas aéreas as quais podemos agregar para contar o número de rotas para cada aeroporto de origem (`origin`).

Entretanto, a base de dados `routes` não inclui as _localizações_ dos aeroportos! Para ampliar os dados `routes` com localizações, precisaremos de uma transformação de dados: procurar (`lookup`). A transformação `lookup` tira um valor de uma coluna em uma base de dados inicial e a usa como _chave_ e procura por alguma informação relacionada em outra tabela. Nesse caso, queremos relacionar o aeroporto em `origin` na nossa base de dados `routes` com o campo `iata` dentro de `airports` e extrair as coordenadas correspondentes pelas colunas `latitude` e `longitude`.

In [39]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending')
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_Quais aeroportos nos EUA tem o maior número de decolagens?_

Agora que podemos ver os aeroportos, podemos querer interagir com eles para entender melhor a estrutura do tráfego aéreo. Podemos adicionar uma camada de marcação regra (`rule`) que representa os caminhos dos aeroportos de `origin` para os aeroportos de destino `destination`, isso requer duas transformações `lookup` para recuperar as coordenadas de cada ponto. Ademais, podemus usar a seleção único (`single`) para filtrar essas rotas, de forma que apenas voos originados no aeroporto selecionado sejam mostrados.

_Começando pelo mapa estatístico acima, você pode construir uma versão interativa? Sinta-se livre para pular o código abaixo e trabalhar com o mapa interativo primeiro, e pense como você poderia construí-lo por conta própria!_

In [42]:
# interactive selection for origin airport
# select nearest airport to mouse cursor
origin = alt.selection_single(
    on='mouseover', nearest=True,
    fields=['origin'], empty='none'
)

# shared data reference for lookup transforms
foreign = alt.LookupData(data=airports, key='iata',
                         fields=['latitude', 'longitude'])
    
alt.layer(
    # base map of the United States
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    # route lines from selected origin airport to destination airports
    alt.Chart(flights).mark_rule(
        color='#000', opacity=0.35
    ).transform_filter(
        origin # filter to selected origin only
    ).transform_lookup(
        lookup='origin', from_=foreign # origin lat/lon
    ).transform_lookup(
        lookup='destination', from_=foreign, as_=['lat2', 'lon2'] # dest lat/lon
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        latitude2='lat2',
        longitude2='lon2',
    ),
    # size airports by number of outgoing routes
    # 1. aggregate flights-airport data set
    # 2. lookup location data from airports data set
    # 3. remove Puerto Rico (PR) and Virgin Islands (VI)
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).add_selection(
        origin
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending') # place smaller circles on top
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

C:\Users\souza\AppData\Local\Temp\ipykernel_12332\940681676.py:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  origin = alt.selection_single(
C:\Users\souza\AppData\Local\Temp\ipykernel_12332\940681676.py:45: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.LayerChart(...)

_Passe o mouse sobre o mapa para explorar a rede de voos!_

## Mapas coropléticos

Um [mapa coroplético](https://en.wikipedia.org/wiki/Choropleth_map) usa regiões sombreadas ou texturizadas para visualizar os valores dos dados. Mapas de símbolos dimensionados são comumente  mais precisos para leitura, pois as pessoas tendem a ser melhores em estimar diferença proporcional entre áreas de círculos à tonalidades de cor. No entanto, mapas coropléticos são populares na prática e especialmente úteis quando muitos símbolos ficam visualmente sobrecarregados.

Por exemplo, enquanto nos EUA têm apenas 50 estados, existem milhares de condados dentro desses estados. Vamos construir um mapa coroplético da taxa de desemprego por condado durante o ano de recessão de 2008. Em alguns casos, arquivos de entrada no formato GeoJSON ou TopoJSON podem incluir dados estatísticos que podemos visualizar diretamente. Porém, neste caso, temos dois arquivos: um arquivo TopoJSON que inclui os limites dos condados (`usa`) e um arquivo de texto separado com as estátisticas de desemprego:

In [43]:
unemp = data.unemployment.url
unemp

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/unemployment.tsv'

Para integrar nossas fontes de dados, nós precisaremos novamente usar a transformação `lookup`, aumentando os dados `geoshape` baseados em TopoJSON com as taxas de desemprego. Em seguida, podemos criar um mapa que inclua uma codificação para cor `(color)` para o campo taxa (`rate`) obtido na busca.

In [44]:
alt.Chart(alt.topo_feature(usa, 'counties')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).encode(
    alt.Color('rate:Q',
              scale=alt.Scale(domain=[0, 0.3], clamp=True), 
              legend=alt.Legend(format='%')),
    alt.Tooltip('rate:Q', format='.0%')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

*Examine as taxas de desemprego por condado. Valores mais altos em Michigan podem estar relacionados a industria automotiva. Condados nas [grandes planícies](https://en.wikipedia.org/wiki/Great_Plains) e nos estados montanhosos exibem tanto taxas baixas **quanto** altas. Essa variação é significativa ou pode ser um [artefato de tamanhos amostrais menores](https://medium.com/@uwdata/surprise-maps-showing-the-unexpected-e92b67398865)? Para explorar mais, tente mudar o domínio da escala superior (por exemplo, para `0.2`) para ajustar o mapeamento de cores.*

Uma preocupação central para os mapas coropléticos é a escolha de cores. No exemplo acima, usamos o esquema padrão do Altair amarelo-verde-azul `(yellowgreenblue)` para mapas de calor. Abaixo, comparamos outros esquemas de cores, incluindo: 
 - Um esquema sequencial de tom único `teals`, que varia apenas em luminância.
 - Um esquema sequencial multicolorido `viridis`, que varia em luminância e matiz.
 - Um esquema divergente `blueorange`, que utiliza um ponto médio em branco. 

In [45]:
# utility function to generate a map specification for a provided color scheme
def map_(scheme):
    return alt.Chart().mark_geoshape().project(type='albersUsa').encode(
        alt.Color('rate:Q', scale=alt.Scale(scheme=scheme), legend=None)
    ).properties(width=305, height=200)

alt.hconcat(
    map_('tealblues'), map_('viridis'), map_('blueorange'),
    data=alt.topo_feature(usa, 'counties')
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).configure_view(
    stroke=None
).resolve_scale(
    color='independent'
)

alt.HConcatChart(...)

_Quais esquemas de cores você achou mais eficazes? Por que isso acontece? Modifique os mapas acima para usar outros esquemas disponíveis, conforme descrito em [Documentação de esquema de cores do Vega](https://vega.github.io/vega/docs/schemes/)._

## Projeções Cartográficas

Agora que temos alguma experiência criando mapas, vamos analisar mais de perto as projeções cartográficas. Como explicado pela [Wikipedia](https://en.wikipedia.org/wiki/Map_projection),

> _Todas as projeções de mapas necessariamente distorcem a superfície de alguma forma. Dependendo do objetivo do mapa, algumas distorções são aceitáveis e outras não; portanto, existem diferentes projeções de mapa para preservar algumas propriedades do corpo esférico em detrimento de outras propriedades._

Algumas das propriedades que podemos considerar incluem:

- _Área_: A projeção distorce o tamanho das regiões?
- _Rumo_ : Uma linha reta corresponde a uma direção de deslocamento constante?
- _Distância_: Linhas de igual comprimento correspondem a distâncias iguais no globo?
- _Forma_: A projeção preserva as relações espaciais (ângulos) entre os pontos? 

Selecionar uma projeção apropriada depende, portanto, do propósito do mapa. Por exemplo, se estamos avaliando o uso da terra e sua  extensão for importante, podemos escolher uma projeção que preserve as áreas. Se quisermos visualizar ondas de choque de um terremoto, podemos focar o mapa no epicentro do terremoto e preservar as distâncias a partir desse ponto. Ou, se quisermos ajudar na navegação, a preservação do rumo e da forma pode ser mais importante.

Também podemos caracterizar as projeções em termos da _superfície de projeção_. Por exemplo, as projeções cilíndricas projetam pontos de superfície da esfera em torno de um cilindro; o cilindro "desenrolado" então fornece nosso mapa. Como descrevemos abaixo, podemos alternativamente projetar na superfície de um cone (projeções cônicas) ou diretamente em um plano liso (projeções azimutais).

Vamos primeiro desenvolver nossa intuição interagindo com uma variedade de projeções! **[abra o capítulo online de projeções cartográficas do Vega-Lite](https://observablehq.com/@vega/vega-lite-cartographic-projections).** Use os controles dessa página para selecionar uma projeção e explorar parâmetros de projeção, como `escala` e translação x/y (panorama). os controles da rotação ([yaw,pitch,roll](https://en.wikipedia.org/wiki/Aircraft_principal_axes)) determinam a orientação do globo em relação à superfície sendo projetada.

### Um Tour pelos Tipos Específicos de Projeção 

As [**Projeções Cilíndricas**](https://en.wikipedia.org/wiki/Map_projection#Cylindrical) mapeiam a esfera em torno de um cilindro, para então desenrolar o cilindro. Se o eixo principal do cilindro está orientado de norte para sul, meridianos são mapeados como linhas retas. As projeções [pseudo-cilíndricas](https://en.wikipedia.org/wiki/Map_projection#Pseudocylindrical) representam o meridiano central como uma linha reta, com outros meridianos "curvando-se" para longe do centro.

In [46]:
minimap = map.properties(width=225, height=225)
alt.hconcat(
    minimap.project(type='equirectangular').properties(title='equirectangular'),
    minimap.project(type='mercator').properties(title='mercator'),
    minimap.project(type='transverseMercator').properties(title='transverseMercator'),
    minimap.project(type='naturalEarth1').properties(title='naturalEarth1')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [Equirretangular](https://en.wikipedia.org/wiki/Equirectangular_projection)(`equirectangular`): Escala os valores de latitude `(lat)` e longitude `(lon)` diretamente
- [Mercador](https://en.wikipedia.org/wiki/Mercator_projection) (`Mercator`): Projeta sobre um cilindro, usando a `lon` diretamente, mas submetendo a `lat` a uma transformação não linear. As linhas retas preservam os rumos constantes ([Linhas de Rumo](https://en.wikipedia.org/wiki/Rhumb_line)), fazendo essa projeção mais adequada para navegação. No entanto, as áreas nas regiões mais ao norte ou sul podem ser bem distorcidas.
- [Mercador Transversal](https://en.wikipedia.org/wiki/Transverse_Mercator_projection) (`transverseMercator`): Uma projeção mercator mas com o cilindro de limite rotacionado para uma eixo transversal. Enquanto a projeção mercador padrão tem maior precisão ao longo do equador, a projeção mercador transversal é mais precisa ao longo do meridiano central.
- [Terra Natural](https://en.wikipedia.org/wiki/Natural_Earth_projection) (`naturalEarth1`): Uma projeção pseudo-cilíndrica projetada para mostrar a Terra inteira em uma única visão.
<br/><br/>

As [**Projeções Cônicas**](https://en.wikipedia.org/wiki/Map_projection#Conic) mapeiam a esfera em torno de um cone e, em seguida, desenrolam o cone para um plano. As projeções são configuradas por dois _paralelas padrão_, que determinam onde o cone intersecta o globo.

In [47]:
minimap = map.properties(width=180, height=130)
alt.hconcat(
    minimap.project(type='conicEqualArea').properties(title='conicEqualArea'),
    minimap.project(type='conicEquidistant').properties(title='conicEquidistant'),
    minimap.project(type='conicConformal', scale=35, translate=[90,65]).properties(title='conicConformal'),
    minimap.project(type='albers').properties(title='albers'),
    minimap.project(type='albersUsa').properties(title='albersUsa')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [Cônica de Áreas Iguais](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection) (`conicConformal`): Projeção cônica que preserva a área. A forma e a distância não são preservadas, mas a precisão é razoavelmente boa dentro dos paralelos padrão.
- [Cônica Equidistantes](https://en.wikipedia.org/wiki/Albers_projection) (`conicEqualArea`): projeção cônica que preserva a distância ao longo dos meridianos e paralelas padrão.
- [Cônica Conformal](https://en.wikipedia.org/wiki/Equidistant_conic_projection) (`conicEquidistant`): projeção cônica que preserva a forma (ângulos locais), mas nem a área nem distância.
- [Albers](https://en.wikipedia.org/wiki/Albers_projection) (`albers`): Uma variante da projeção cônica que iguala a área projetada com as paralelas padrão otimizadas para a criação de mapas dos Estados Unidos.
- [Albers USA](https://en.wikipedia.org/wiki/Albers_projection) (`albersUsa`): Uma projeção híbrida para os 50 estados dos EUA. Essa projeção une três projeções Albers com parâmetros diferentes para os EUA continental, Alasca e Havai.
<br/><br/>

As [**Projeções Azimutais**](https://en.wikipedia.org/wiki/Map_projection#Azimuthal_%28projections_onto_a_plane%29) mapeiam a esfera diretamente para um plano.

In [28]:
minimap = map.properties(width=180, height=180)
alt.hconcat(
    minimap.project(type='azimuthalEqualArea').properties(title='azimuthalEqualArea'),
    minimap.project(type='azimuthalEquidistant').properties(title='azimuthalEquidistant'),
    minimap.project(type='orthographic').properties(title='orthographic'),
    minimap.project(type='stereographic').properties(title='stereographic'),
    minimap.project(type='gnomonic').properties(title='gnomonic')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [Azimutal de Área Igual](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection) (`azimuthalEqualArea`): Projeta áreas com precisão em todas as partes do globo, mas não preserva a forma (ângulos locais).
- [Azimutal Equidistante](https://en.wikipedia.org/wiki/Azimuthal_equidistant_projection) (`azimuthalEquidistant`): Mantém as distâncias proporcionais entre o centro da projeção e todos os outros pontos no globo.
- [Ortográfica](https://en.wikipedia.org/wiki/Orthographic_projection_in_cartography) (`orthographic`): Projeta um hemisfério visível em um plano distante, simulando aproximadamente a visão da Terra a partir do espaço.
- [Estereográfica](https://en.wikipedia.org/wiki/Stereographic_projection) (`stereographic`): Preserva a forma, mas não mantém nem a área nem a distância.
- [Gnomônica ](https://en.wikipedia.org/wiki/Gnomonic_projection) (`gnomonic`): Projeta a superfície da esfera diretamente em um plano tangente. Os [grandes círculos](https://en.wikipedia.org/wiki/Great_circle) em torno da Terra são projetados em linhas retas, mostrando o menor caminho entre pontos.
<br/><br/>

## Epílogo: Manipulando Dados Geográficos 

Os exemplos acima foram extraídos da coleção vega-datasets, incluindo dados geométricos (TopoJSON) e tabulares (como aeroportos e taxas de desemprego).Um desafio comum para começar a visualização geográfica é coletar os dados necessários para sua tarefa. Existem diversos provedores de dados disponíveis, incluindo serviços como: [United States Geological Survey](https://www.usgs.gov/products/data/all-data) e [U.S. Census Bureau](https://www.census.gov/data/datasets.html).

Em vários casos, você pode já possuir dados com um componente geográfico, mas precisar de medidas ou geometrias adicionais. Para ajudar você a começar, aqui está um fluxo de trabalho:

1. Visite [Natural Earth Data](http://www.naturalearthdata.com/downloads/) e pesquise para selecionar dados por regiões e resuloções de interesse. Baixe o(s) arquivo(s) zip correspondente(s).

2. Vá para [MapShaper](https://mapshaper.org/) e arraste seu arquivo zip para a página. Edite os dados como quiser, e em seguida, "exporte" os arquivos TopoJSON ou GeoJSON gerados.

3. Carregue os dados exportados do MapShaper para usar no Altair!

Claro, muitas outras ferramentas &ndash; de código aberto e proprietárias &ndash; existem para trabalhar com dados geográficos. Para mais sobre manipulação de dados geográficos e criação de mapas, veja a série de tutorial do Mike Bostock [Command-Line Cartography](https://medium.com/@mbostock/command-line-cartography-part-1-897aa8f8ca2c).

## Resumo

Até este ponto, nós apenas arranhamos a superfície do oceano de criação de mapas. _(Você não esperava que um único capítulo transmitisse séculos de estudo, né?)_ Por exemplo, nós deixamos de lado tópicos como [_cartogramas_](https://en.wikipedia.org/wiki/Cartogram) e a [_representação de relevos_](https://en.wikipedia.org/wiki/Topography) &mdash; como no brilhante livro de Imhof [_Cartographic Relief Presentation_](https://books.google.com/books?id=cVy1Ms43fFYC). Porém, agora você deveria estar bem preparado para criar uma vasta gama de geo-visualização. Para mais, o livro de MacEachren [_How Maps Work: Representation, Visualization, and Design_](https://books.google.com/books?id=xhAvN3B0CkUC) oferece uma visão geral valiosa da criação de mapas da perspectiva da visualização de dados.